# ORCA Predictions

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()
from IPython.display import HTML

First let us plot the environment prediction without any ego agent (i.e. robot) in the scene.

In [2]:
import torch

from mantrap.agents import IntegratorDTAgent
from mantrap.constants import *
from mantrap.environment import ORCAEnvironment

ego_kwargs = {"position": torch.tensor([9.9, 9.9]), "velocity": torch.zeros(2)}
env = ORCAEnvironment(IntegratorDTAgent, ego_kwargs=ego_kwargs, dt=0.2)
env.add_ado(position=torch.tensor([5, 0]), velocity=torch.tensor([-0.1, -0.4]), goal=torch.zeros(2))
env.add_ado(position=torch.tensor([-4, 2]), velocity=torch.tensor([1, -1]), goal=torch.tensor([4.0, -2.0]))
env.add_ado(position=torch.tensor([-2, -2]), velocity=torch.tensor([1, 1]), goal=torch.tensor([2.0, 2.0]))

HTML(env.visualize_prediction_wo_ego(t_horizon=20, interactive=True, enforce=True))

Now we add a robot ego trajectory in the scene, but keep it in distance from the other agents and do not move it. In this scenario the conditioned predicted trajectories should not differ much from the case above, without having an agent in the scene at all.

In [3]:
ego_trajectory = env.ego.unroll_trajectory(controls=torch.zeros((10, 2)), dt=env.dt)
HTML(env.visualize_prediction(ego_trajectory=ego_trajectory, interactive=True, enforce=True))

Now let us use the ORCA-based solver, which basically sets up a new environment with the robot being another similar agent instead of seperating between ego and ados. Then the trajectory of the ego is solved (which of course ignores any objectives and constraints other than the ones defined in the ORCA algorithm itself). Also it assumes that all other agents in the scene (ado) behave as being controlled using the ORCA algorithm.

In [4]:
from mantrap.solver import ORCASolver

ego_kwargs = {"position": torch.tensor([1.9, 0.9]), "velocity": torch.zeros(2)}
env = ORCAEnvironment(IntegratorDTAgent, ego_kwargs=ego_kwargs, dt=0.2)
env.add_ado(position=torch.tensor([5, 0]), velocity=torch.tensor([-0.1, -0.4]), goal=torch.zeros(2))

solver = ORCASolver(env, goal=torch.tensor([0, -1]), t_planning=1)
ego_trajectory, _ = solver.solve(time_steps=10)
HTML(env.visualize_prediction(ego_trajectory=ego_trajectory, interactive=True, enforce=True))